In [1]:
##### Packages and Libraries #####

# Clear all variables
%reset -f

# Display pretty table for dataframe
from IPython.display import display

import numpy as np
import pandas as pd

from datetime import datetime, date
from math import sqrt, ceil, floor

from pandas.plotting import table

from matplotlib import pyplot as plt
from matplotlib.offsetbox import AnchoredText
from mpl_toolkits import mplot3d
from matplotlib import cm
from matplotlib import dates as mdates

# Directory browising and local file storage access
import glob
import os

# Data from web
import requests

import scipy.stats as ss

In [2]:
# Constants and Global Variable
# Security
security = 'AAPL'

In [3]:
# Get daily security price
data_type = 'daily'
# get latest data file for specified security
file_name = max(glob.iglob('data/{}_{}_*.csv'.format(security, data_type)), key=os.path.getctime)
price_df = pd.read_csv(file_name, index_col=0, parse_dates=True, infer_datetime_format=True)
# Remove any duplicate date data. Sometime data received has duplicate dates.
# Assuming the last duplicate is the updated/corrected data, keep the last duplicated row.
price_df = price_df[~price_df.index.duplicated(keep='last')]

In [4]:
# Get Corporate Actions
try:
    file_name = max(glob.iglob('data/{}_actions.csv'.format(security)), key=os.path.getctime)
    actions_df = pd.read_csv(file_name, index_col=0, parse_dates=True, infer_datetime_format=True)

    # latest dividend
    dividend = actions_df[actions_df.action == 'DIVIDEND'].head(1).value.values[0]

    print('{} Recent Corporate Actions and Dividends'.format(security))
    display(actions_df.join(price_df.Close, how='left').head().style.format_index(lambda s: s.strftime("%Y-%m-%d")).format(formatter={'value': '${:,.2f}', 'Close': '${:,.2f}'}))
except:
    dividend = 0.00
    print('{} Corporate Action data file not found.'.format(security))

AAPL Recent Corporate Actions and Dividends


,action,value,Close
2021-05-07,DIVIDEND,$0.22,$nan
2021-02-05,DIVIDEND,$0.20,$nan
2020-11-06,DIVIDEND,$0.20,$nan
2020-08-31,SPLIT,$0.25,$nan
2020-08-07,DIVIDEND,$0.20,$nan


In [5]:
# Get Corporate Activity Calendar
try:
    file_name = max(glob.iglob('data/{}_calendar.csv'.format(security)), key=os.path.getctime)
    calendar_df = pd.read_csv(file_name, index_col=0, parse_dates=True, infer_datetime_format=True)
    calendar_df = calendar_df.T
    calendar_df['Earnings Date'] = pd.to_datetime(calendar_df['Earnings Date'], infer_datetime_format=True, utc=True)
    calendar_df.set_index('Earnings Date', inplace=True)
    calendar_df.index = calendar_df.index.tz_convert('America/New_York')
    # change data type from string to numeric
    calendar_df = calendar_df.apply(pd.to_numeric)
    earnings_disperson = 100.0 * (calendar_df['Earnings High'] - calendar_df['Earnings Low']) / calendar_df['Earnings Average']
    revenue_disperson = 100.0 * (calendar_df['Revenue High'] - calendar_df['Revenue Low']) / calendar_df['Revenue Average']
    print('{} Next Earnings and Revenue Estimates'.format(security))
    print('\tEarnings Dispersion: {:,.2f}%, Revenue Disperson: {:,.2f}%'.format(earnings_disperson.values[0], revenue_disperson.values[0]))
    display(calendar_df.style.format(formatter={'Earnings Average': '${:,.2f}', 'Earnings Low': '${:,.2f}', 'Earnings High': '${:,.2f}', 'Revenue Average': '${:,.0f}', 'Revenue Low': '${:,.0f}', 'Revenue High': '${:,.0f}'}))
except:
    calendar_df = None
    print('{} Corporate Activity Calendar data file not found.'.format(security))

AAPL Next Earnings and Revenue Estimates
	Earnings Dispersion: 22.14%, Revenue Disperson: 12.73%


,Earnings Average,Earnings Low,Earnings High,Revenue Average,Revenue Low,Revenue High
Earnings Date,,,,,,
2022-04-25 20:00:00-04:00,$1.31,$1.15,$1.44,"$89,996,500,000","$84,994,900,000","$96,451,000,000"
2022-05-01 20:00:00-04:00,$1.31,$1.15,$1.44,"$89,996,500,000","$84,994,900,000","$96,451,000,000"


In [6]:
# Get Earnings History
try:
    file_name = 'data/{}_earnings_hist.csv'.format(security)
    earnings_df = pd.read_csv(file_name, index_col=0, parse_dates=True, infer_datetime_format=True)
    earnings_df.index = pd.to_datetime(earnings_df.index, utc=True, infer_datetime_format=True)
    earnings_df.index = earnings_df.index.tz_convert('America/New_York')
    earnings_df.dropna(axis=0, how='all', inplace=True)
    earnings_df.index = earnings_df.index.date
    # Add Closing Price of the day for display only
    display(earnings_df.join(price_df.Close, how='left').head().style.format_index(lambda s: s.strftime("%Y-%m-%d")).format(formatter={'EPS Estimate': '${:,.2f}', 'Reported EPS': '${:,.2f}', 'Surprise(%)': '{:,.2%}', 'Close': '${:,.2f}'}))
except:
    earnings_df = None
    print('{} Earnings History data file not found'.format(security))

,EPS Estimate,Reported EPS,Surprise(%),Close
2022-10-26,$1.26,$nan,nan%,$nan
2022-07-28,$1.16,$1.20,3.27%,$157.35
2022-04-28,$1.43,$1.52,6.44%,$163.64
2022-01-27,$1.89,$2.10,11.17%,$159.22
2021-10-28,$1.24,$1.24,0.32%,$152.57


In [7]:
# Get TipRanks Earnings History and Price Change
try:
    file_name = 'data/{}_earnings_hist_tipranks.csv'.format(security)
    tipranks_earnings_df = pd.read_csv(file_name, index_col=0, parse_dates=True, infer_datetime_format=True)
    print('{} TipRanks Earnings History and Price Response Data'.format(security))
    display(tipranks_earnings_df.head())
except:
    tipranks_earnings_df = None
    print('{} TipRanks Earnings History data file not found'.format(security))

AAPL TipRanks Earnings History and Price Response Data


,report_date,fiscal_yr,fiscal_qtr,eps_forecast,eps_actual,eps_prev_yr,eps_yoy_chg,eps_yoy_pct,price_1d_before,price_1d_after,price_pct_change
0,2022-11-03,2022,4,1.26,NaN,1.24,NaN,NaN,NaN,NaN,NaN
1,2022-07-28,2022,3,1.16,1.20,1.30,-0.10,-7.69,157.13,162.28,3.28
2,2022-04-28,2022,2,1.42,1.52,1.40,0.12,8.57,163.17,157.20,-3.66
3,2022-01-27,2022,1,1.90,2.10,1.68,0.42,25.00,158.56,169.63,6.98
4,2021-10-28,2021,4,1.24,1.24,0.73,0.51,69.86,151.72,148.96,-1.82


In [8]:
# Calculate Log Change from Previous Day (these are additive over longer period)
price_df['LogChng'] = np.log(price_df.Close / price_df.Close.shift(1))

In [9]:
# Add columns that count number of days before earnings days and after earnings days
# Save the index name and reset index
index_name = price_df.index.name
price_df.reset_index(inplace=True)

In [10]:
# Find rows in price data corresponding to earnings date
# Use TipRanks earnings table if Yahoo! earnings table not available
if earnings_df is not None:
    earnings_days = np.where(price_df.Date.isin(earnings_df.index))[0].tolist()
elif tipranks_earnings_df is not None:
    earnings_days = np.where(price_df.Date.isin(tipranks_earnings_df.report_date))[0].tolist()
else:
    earnings_days = []
    
# Add the lowest and highest index of price_df
# Need this to know the calculation boundaries for first and last earnings days, 
# otherwise for loop will get index error.
if len(earnings_days) != 0:
    earnings_days.extend([price_df.index.min(), price_df.index.max()])
    earnings_days.sort()

print(earnings_days)

[0, 29, 91, 154, 216, 251]


In [11]:
# Add days before earnings and days after earnings
for i in range(1, len(earnings_days)-1):
    price_df.loc[earnings_days[i-1]+1:earnings_days[i], 'pre_earn'] = (price_df.Date - price_df.loc[earnings_days[i]].Date).dt.days
    price_df.loc[earnings_days[i]:earnings_days[i+1], 'post_earn'] = (price_df.Date - price_df.loc[earnings_days[i]].Date).dt.days
    
# Make sure 0 days line up correctly with earnings dates
for i in range(1, len(earnings_days)-1):
    display(price_df.loc[earnings_days[i]-2:earnings_days[i]+2])

,Date,High,Low,Open,Close,Volume,Adj Close,LogChng,pre_earn,post_earn
27,2021-10-26,150.839996,149.009995,149.330002,149.320007,60893400.0,148.487946,0.004564,-2.0,NaN
28,2021-10-27,149.729996,148.490005,149.360001,148.850006,56094900.0,148.020569,-0.003153,-1.0,NaN
29,2021-10-28,153.169998,149.720001,149.820007,152.570007,100077900.0,151.719849,0.024684,0.0,0.0
30,2021-10-29,149.940002,146.410004,147.220001,149.800003,124953200.0,148.965256,-0.018322,-90.0,1.0
31,2021-11-01,149.699997,147.800003,148.990005,148.960007,74588300.0,148.129944,-0.005623,-87.0,4.0


,Date,High,Low,Open,Close,Volume,Adj Close,LogChng,pre_earn,post_earn
89,2022-01-25,162.759995,157.020004,158.979996,159.779999,115798400.0,159.121536,-0.011450,-2.0,89.0
90,2022-01-26,164.389999,157.820007,163.500000,159.690002,108275300.0,159.031921,-0.000563,-1.0,90.0
91,2022-01-27,163.839996,158.279999,162.449997,159.220001,121954600.0,158.563858,-0.002948,0.0,0.0
92,2022-01-28,170.350006,162.800003,165.710007,170.330002,179935700.0,169.628067,0.067451,-90.0,1.0
93,2022-01-31,175.000000,169.509995,170.160004,174.779999,115541600.0,174.059723,0.025790,-87.0,4.0


,Date,High,Low,Open,Close,Volume,Adj Close,LogChng,pre_earn,post_earn
152,2022-04-26,162.339996,156.720001,162.250000,156.800003,95623200.0,156.352783,-0.038043,-2.0,89.0
153,2022-04-27,159.789993,155.380005,155.910004,156.570007,88063200.0,156.123444,-0.001468,-1.0,90.0
154,2022-04-28,164.520004,158.929993,159.250000,163.639999,130216800.0,163.173264,0.044166,0.0,0.0
155,2022-04-29,166.199997,157.250000,161.839996,157.649994,131747600.0,157.200348,-0.037292,-90.0,1.0
156,2022-05-02,158.229996,153.270004,156.710007,157.960007,123055300.0,157.509476,0.001965,-87.0,4.0


,Date,High,Low,Open,Close,Volume,Adj Close,LogChng,pre_earn,post_earn
214,2022-07-26,153.089996,150.800003,152.259995,151.600006,55138700.0,151.389725,-0.008866,-2.0,89.0
215,2022-07-27,157.330002,152.160004,152.580002,156.789993,78620700.0,156.572510,0.033662,-1.0,90.0
216,2022-07-28,157.639999,154.410004,156.979996,157.350006,81378700.0,157.131744,0.003565,0.0,0.0
217,2022-07-29,163.630005,159.500000,161.240005,162.509995,101786900.0,162.284576,0.032267,NaN,1.0
218,2022-08-01,163.589996,160.889999,161.009995,161.509995,67829400.0,161.285965,-0.006172,NaN,4.0


In [12]:
# Earnings day price data
price_df.loc[np.where(price_df.Date.isin(tipranks_earnings_df.report_date))]

,Date,High,Low,Open,Close,Volume,Adj Close,LogChng,pre_earn,post_earn
29,2021-10-28,153.169998,149.720001,149.820007,152.570007,100077900.0,151.719849,0.024684,0.0,0.0
91,2022-01-27,163.839996,158.279999,162.449997,159.220001,121954600.0,158.563858,-0.002948,0.0,0.0
154,2022-04-28,164.520004,158.929993,159.250000,163.639999,130216800.0,163.173264,0.044166,0.0,0.0
216,2022-07-28,157.639999,154.410004,156.979996,157.350006,81378700.0,157.131744,0.003565,0.0,0.0


> Euan Sinclair, Option Trading, Pricing and Volatility Strategies and Techniques, Wiley, 2010.

## 2. Introduction to Options

### Uses of Options

#### Hedging a long Position with a Put

Although the theory of option pricing is based on the idea that we can replicate an option position by dynamically trading the underlying, this is just an approximation. In practice the subtle distinctions between option behavior and the behavior of the replicating strategy are where professional traders can make money.

#### Volatility Trading

By trading options and dynamically hedging them with the underlying, a trader can take a position that is dependent only on the difference in volatility implied by the option market and the volatility that actually occurs in the underlying during the lifetime of the option.

### Market Structure

#### Margin

**Strategy-Based Margin**

*Long Calls or Puts.* An option must be paid in full.

*Short Calls or Puts.* The margin is the greater of:

* The proceeds of the sale plus 20% of the value of the underlying minus the amount of the option is OTM; or
* The proceeds of the sale plus 10% of the value of the underlying.

<u>Example:</u> Sold 105C @ \\$3.00 when underlying @ \\$100, the margin requirements:

1. \\$3.00 x 100 + 20% x \\$100 x 100 - (\\$105 - \\$100) x 100 = $1,800
2. \\$3.00 x 100 + 10% x \\$100 x 100 = \\$1,300

Total margin = \\$1,800 - \\$300 = \\$1,500

**Portfolio Margin**

*Stock* positions are tested with price changes of up and down 15%. *Small cap broad-based indices* are tested with 10% price changes, and *large cap broad-based indices* are tested with a move of down 6% and up 8%. The range is divided into 8 equal units, and the loss on the position is calculated at each of the eight points and the two high and low points. There are several adjustments applied:

* Minimum margin of $37.50
* "Similar" classes such as SPX and OEX options are assumed to have offsetting risks.
* In addition to the underlying price shift, a volatility shift can also be applied.

## 3. Arbitrage Bounds for Option Prices

The principle of no arbitrage is probably the closest thing finance has to a fundamental law. Also, known as the "law of one price," this states that if two securities are same, then they will have same price. If they do not, then arbitrageurs will trade the securities and make a risk-free profit. The law of no arbitrage is somewhat self-fulfilling.

Sometimes what appears to be an arbitrage opportunity is merely a situation with larger than anticipated transaction costs or execution risk (delay in second offsetting trade).

A common form of arbitrage is where two securities are non identical, but have a well-defined financial relationship to construct a risk free portfolio from combination (Example: Forward contract).

*Forward Contract*

A stock with a current price of $S$, want to sell at a time $T$ in future. What is the price $F$, at which we would be indifferent about agreeing to sell it?

$$
F = S \cdot e^{r T}
$$

$r$ is the risk-free interest rate. This equation can be proven by assuming it does not hold and showing that this leads to possible arbitrage profits. There are a few assumptions to note:

1. The underlying must be an asset we can actually trade, i.e. ability to buy it and sell it short. If only one of these two conditions hold, we will have an inequality rather than an equality.
2. The interest rate to borrow and to lend are same. If this is not the case, we would arrive at no arbitrage band, instead of at single value.
3. Holding the underlying gives no addiitonal benefit, e.g. dividend/interest/shareholder benefits. If underlying is paid a dividend. equation will be adjusted to account for present value of dividend.

$$
F = (S - PV_{dividend}) \cdot e^{r T}
$$

4. Holding the underlying doesn't incur storage costs (commodities). These costs would behave like a *negative* dividend.
5. Interest rate is paid by an entity that will not default.
6. Interest rate is constant. This is important when we are considering the difference between forward contracts and futures. Futures are marked to market each day and the amount of collateral a trader need to post is hence adjusted daily. If interest rates and underlyings are positively correlated, futures will be worth slightly nore than forward contracts. Practically in every case we can assume futures are priced as forwards.
7. The future contrcat's price is *not* the expected price in the future. It is merely the price that allows no arbitrage.

This is an example of arbitrage through *static replication*. Option pricing is based on the idea of *dynamic replication*, where a position needs to be rebalanced to maintain riskless nature of arbitrage.

### American Options compared to European Options

An American option is always worth at least as much as a European option. If we ever find an American option trading for less than an equivalent European option we can create risk-free profit by buying American option, selling European option, then holding the position until expiration. Capital letters denote American options and lowercase letters European options, $ c \le C $ for calls and $ p \le P $ for puts.

### Absolute Maximum and Minimum Values

#### The Maximum Value of a Call

A call cannot cost more than the underlying, $ c \le S $, the right to purchase an item cannot be worth more than the item itself.

#### The Maximum Value of a Put

An American put can never be worth more than the strike price, $ P \le X $, the most value a put can ever have is when underlying goes to zero, the payoff is equal to the strike. For a European option, it will take maximum value only at expiry. So we need to discount this amount, that gives lesser bound.

$$ p \le X \cdot e^{-rt} $$

#### Minimum Value for a Call Option

The lower bound for a European call option is:

$$ c \ge S - X \cdot e^{-rt} $$

#### Minimum Value for a Put Option

The lower bound for a European put option is:

$$ p \ge X \cdot e^{-rt} - S $$

Both American calls and puts have lower bounds given by their intrinsic values.

$$ C \ge Max(0, S - X) $$
$$ P \ge Max(0, X - S) $$

European options can trade below intrinsic value due to the effects of discounting. This is never the case for American options. Unless we are at expiry or interest rates are zero,

$$ S - X \cdot e^{-rt} > S - X $$

American options are always at least as valuable as European options, the lower bound for the European call must also apply to the American option.

$$ C = Max(0, S - X \cdot e^{-rt}) $$

We should never exercise a call on a non-dividend paying stock because an ITM option has a minimum value of $ S - X \cdot e^{-rt} $ that is greater than the intrinsic value of $ S - X $. If a trader wants to realize the profits from a deep ITM call position, he will be better off selling the option that exercising it.

### Calls with different Strikes

Consider two strikes, $ X_2 $ and $ X_1 $, where $ X_2 \gt X_1 $. Now construct two portfolios: the first (Portfolio 1) consisting of a long position in $ c_1 = c(X_1) $ and a short position in $ c_2 = c(X_2) $, and the second (Portfolio 2) a cash position worth $ (X_2 - X_1) \cdot e^{-rt} $.

The payoff for Portfolio 1 is always positive including the initial value $ c(X_1) \gt c(X_2) $. The Portfolio 2 is always worth at least as much as the first.

$$ c(X_1) - c(X_2) \le (X_2 - X_1) \cdot e^{-rt} $$

For American options, if both calls were exercise early, $ S \gt X_2 $, we could obtain $ X_2 - X_1 $ in cash and could then invest this. This would make the call spread portfolio worth more than the cash portfolio. To avoid this we need to structure the cash portfolio so that it has a current value of $ X_2 - X_1 $.

$$ C(X_1) - C(X_2) \le X_2 - X_1 $$

### Put with different Strikes

Construct two portfolios. The first is short a European put at $ X_1 $, and a long position in a European put at $ X_2 $. The seconf portfolio consists of cash, $ (X_2 - X_1) \cdot e^{-rt} $.

The final payoff on the put portfolio is always non-negative, i.e. current value must also be non-negative $ p(X_2) \ge p(X_1) $. European puts with a higher strike price are always worth at least as much as those with lower strike price. The possibility of early exercise with American option does not change the result $ P(X_2) \ge P(X_1) $.

The second portfolio is always worth at least as much as the first. This gives upper bound on the value of the European put spread.

$$ p(X_2) - p(X_1) \le (X_2 - X_1) \cdot e^{-rt} $$

For American put spread, the upper bound is given by

$$ P(X_2) - P(X_1) \le (X_2 - X_1) $$

### The "Butterfly" Relationship

Consider European call options with strikes $ X_1 $, $ X_2 $, and $ X_3$, where $ X_3 \gt X_2 \gt X_1 $. The first portfolio consists of $ \alpha $ units of $ c(X_1) $ and $ (1 - \alpha) $ units of $ c(X_3) $.

$$ \alpha = \frac{(X_3 - X_2)}{(X_3 - X_1)} $$

$ \alpha \gt 0 $ because $ X_3 \gt X_2 $ and $ X_3 \gt X_1 $.

The second portfolio consists of middle calls, $ c(X_2) $.

If $S \lt X_1$, the portfolio produces same result. If $X_1 \lt S \lt X_2$ or $X_2 \lt S \lt X_3$, the first portfolio is better because $X_2 > X_1$. When. $S \lt X_1$ the portfolios are equal.

$$ \alpha \cdot c(X_1) + (1 - \alpha) \cdot c(X_3) \ge c(X_2)$$

For American option,

$$ \alpha \cdot C(X_1) + (1 - \alpha) \cdot C(X_3) \ge C(X_2)$$

Same argument also applies to puts.

In the special case where strikes are equally spaced $X_3 - X_2 = X_2 - X_1$, $\alpha = 0.5$. This portfolio is known as a butterfly. For European call butterfly,

$$ c(X_1) + c(X_3) \ge 2c(X_2) $$ or

$$ c(X_1) - 2c(X_2) + c(X_3) \ge 0$$ Butterflies can never be negative.

### Relationships between Calls with Different Expirations

Assume, Expiry $ T_2 \gt T_1 $. At the expiry of the shorter dated option, it will be worth $ Max(0, S(T_1) - X) $. However, longer dated option still has more time remaining so its minimum value is $ Max(0, S(T_1) - X \cdot e^{(-r(T_2 - T_1))}) $ which is greater than the value of shorter-term option. So $ c(T_2) \ge c(T_1) $.

If the undelrying pays dividends, the longer dated option has minimum value $ Max(0, S(T_1) - D - X \cdot e^{-r(T_2 - T_1)}) $. The longer dated option <u>could</u> take a value less than that of the shorter dated option.

For American options, the longer dated call could not be worth less than intrinsic value, so $ C(T_2) \ge C(T_1) $.

### Relationships between Puts with Different Expirations

For European puts, when underlying pays no dividends, the value of shorter dated option is $ Max(0, X - S) $ at expiry. At this time, longer dated option is worth at least $ Max(0, X \cdot e^{-r(T_1 - T_2)} - S) $. In this case the short term option could be worth more. It is related to the reason that we will sometimes decide to exercise an American put early. If we exercise we get the cash early and can invest it. So in this case holding the option hurts us. This argument also applies when dividends are being paid. However for American options it does not hold. Here $ P(T_2) \ge P(T_1) $.

### Put Call Parity

Put call parity is an arbitrage relationship between European calls and puts of the same strike.

$$ c - p = S - X \cdot e^{-rT} $$

Construct a portfolio by buying a put, selling a call with same strike, and buying one share of stock. At expiry, this portfolio must be worth the strike price, thus currently must be worth the discounted value of the strike price.

$$ p - c + S = X \cdot e^{-rT} $$

This relationship has a number of important implications.

1. $ c = S - X \cdot e^{-rT} + p $ Owning a call is like borrowing money to buy stock and also owning a put.

2. For nonzero interest rates, if the option strike is at spot price (S = X), calls are worth more than puts. For call and put to have equal value we need $ S = X \cdot e^{-rT} $ or $ X = S \cdot e^{rT} $. The options must be struck at the forward price.

3. We can replicate the payoff from any position with the other three instruments. For example, a call can be replicated with a put, stock, and cash.

Put call parity does not hold for American options. In order to create a risk-free position we must be assured that short option will not be exercised.

For American options on nondividend paying stocks, the put call parity relationship is 

$$ S - X \le C - P \le S - X \cdot e^{-rT} $$

and, for diividend paying stocks, where PVD is the present value of dividends, is

$$ S - PVD - X \le C - P \le S - X \cdot e^{-rT} $$

### Boxes, i.e. Box Spread

We Own $ c(X_1) $, are short $ c(X_2) $, own $ p(X_2) $, and are short $ p(X_1) $, i.e. we are long the call spread and also long the put spread. This is a box or a box spread.

No matter what happens, this strategy is always worth $ X_2 - X_1 $ at expiry or worth the present value of $ X_2 - X_1 $ at any time.

$$ c(X_1) - c(X_2) + p(X_2) - p(X_1) = (X_2 - X_1) \cdot e^{-rt} $$

A box spread is useful because its risk is entirely dependent on the interest rate.

Traders should be very careful when selling an American box. It may be the counterparty's way of putting on a trade that will profit if a special dividend is paid when an American call will become far more valuable.

### Summary

* American options are worth more than European options.
* Longer dated options are worth more than shorter dated options.
* Calls cannot be worth more than the underlying.
* Puts cannot be worth more than the discounted strike price.
* Call spreads are positive.
* Put spreads are positive.
* Butterflies are positive.
* Put call parity allows us to transform cals and vice versa.
* Boxes are the discounted value of the differnece betwen the strike.

## 4. Pricing Models

A model is a framework for thinking about trades. It need not say anything real about the world.

### General Modeling Principles

* Have we used the correct mathematical or statistical methods?
* Do the variables chosen on which to base our model make sense?
* How should we interpret the results?

#### Choosing the Tools

The value of an option is clearly related to the value of the underlying. So a natural way to price options would be to simulate the underlying or to look at its price distribution. The first way will lead to Monte Carlo or tree-based methods, and the second to the Black-Scholes-Merton world.

#### Using Suitable Variables

To avoid multicolinearity, it is important that the variables we choose are, as much as possible independent of one another.

* Think about possible codependance between the variables.
* Consider if the data that is available is a sensitive measure of the effect you are trying to include.
* Decide if you have left anything important out.

#### Interpretation of Results

To do this a trader needs to have the ability to transplate the results from the particular mathematical form of the solution into something intuitive.

The next step is to use the results of the modeling to gain insights.

### Choice of Dependent Variables

[TODO]: 

> Euan Sinclair, Positional Option Trading: An Advanced Guide, Wiley, 2020

# Introduction

* Find a robust source of edge that is backed by empirical evidence and convincing reasons for its existence.
* Choose the appropriate option structure to monetize the edge.
* Size the position appropriately.
* Always be aware of how much you don't know.

<u>There is often an edge in selling options with high volatilities relative to others on the same underlying.</u>

Let's accept the Efficient Market Hypothesis (EMH) is imperfect enough that it is possible to make money. The economists who study these deviations from perfection classify them into two classes: risk premium and inefficiencies. A **risk premium** is earned as compensation for taking a risk, and if the premium is mispriced, it will be profitable even after accepting the risk. An **inefficiency** is a trading opportunity caused by the market not noticing something. It is often impossible to know whether a given opportunity is a risk premium or an inefficiency. A risk premium can be expected to persist. By contrast, an inefficiency will last only until other people notice it.

The alpha derived from market inefficiencies, as opposed to the beta of exposure to a mispriced risk factor, doesn't last forever. Depending on how easy it is to trade the effect, the half-life of an inefficiency based strategy seems to be between 6 months and 5 years. Sometimes the alpha isn't due to a wrinkle in the financial market. It is due to the costs of processing information.

There is overwhelming evidence that stocks have momentum. Stocks that have outperformed tend to continue outperforming. This has been observed for as long as we have data *Geczy, C., and M. Samnov. 2016. “Two Centuries of Price Return Momentum.” Financial Analysts Journal 72: 32–56*. In the trading world, the evidence for stock momentum is overwhelming. Starting from this fact, design a simple model to measure momentum, e.g. 6-month return. Then sort stocks by this metric and buy the ones that score well.

# Forecast Volatility

Except for a pure arb, which are practically extinct, to get positive expectation we need to make a forecast that is both correct and more correct than the consensus.

* Aim to find problems that are solvable but are hard enough that you will be able to profit from the predictions. Volatility is a perfect candidate for this.

* Actively look for comparable historical situations. What happens when the government shuts down? What is the link between recessions and the stock market? This is a good general principle, but it is also vital if you are looking for catalysts that could lead to volatility explosions. Good periods to be short volatility can often be deduced from financial data alone, but long trades generally need a catalyst, that isn't priced in, to be successful.

* Aim to balance being conservative and reactive.

* Actively look for counterarguments. There are no certianties when predicting the future.

Forget about whether volatility follows a GARCH(1,1) or a T-GARCH(1,2) process; the important observation is that it **clusteres in the short term and mean reverts in the long term**. If the phenomenon is strong enough to trade, it shouldn't be crucial what exact model is used.

## Model-Driven Forecasting and Situational Forecasting

Mispriced risk premia can last for a long time. A business can be built on harvesting risk premia. <u>Inefficiencies aren't likely to be as persistent. These needs to be aggressively traded while they last, and we can assume that they won't last long.</u>

There is also an important classification of trades at the tactical level. Strategy defining the high-level goals and tactics being the methods we use to reach them. <u>Trades are either model driven or event driven.</u>

With a model-driven trade, we have a theoretical model of a situation that lets us calculate a fair value or edge. At any moment, we will have an opinion based on the model. For example, if we have an option pricing model, we can continually generate a theoretical value for all of the options on a given underlying. An event-driven trade is based on a specific unusual situation. Noticing that implied volatility declines after a company releases earnings would be the basis of an event-driven trade.

We can rank stocks with a factor model such as Fama-French-Carhart, or we could buy stocks that has positive earnings surprises.

For a model-driven approach to be effective we need a good model. In the 1980s collecting daily closing prices and calculating volatility from those was enough to give a volatility edge in the option markets. Not it seems like there is not edge left in this volatility arbitrage model.

The process for finding and testing Event-driven trades is very simple. What happens to stocks in the three days after a FED meeting? What does the VIX do on Mondays? Are teeny options overpriced? Most important, trades that are based on specific events or situations can be very profitable. A drawback of event-based trades is that we must wait for the event, and some events don't occur often.

Our focus is finding *situations* where we can do better than the consensus. The ease of finding and manipulating financial data has considerably lessened the efficacy of forecasting volatility using time series models, but measuring and forecasting volatility in this way is still necessary for trade sizing and allocation.

## The GARCH Family and Trading

The simplest forecasting model is to assume that the volatility over the next N days will be the same as it was over the previous N days. Mathematically, $\sigma_t^2 = \sigma_{t-1}^2$. This has two major problems. First is the "windowing" effect where a large single return affects the volatility calculation for N days, then drops out of the sample. This creates jumps in the volatility measurements and hence the forecast. A bigger problem is that this method doesn't take volatility clustering into account. Periods of exceptionally high or low volatility will persist for only a short time.

The exponentially weighted moving average (EWMA) model takes this into account. $\sigma_t^2 = \lambda\sigma_{t-1}^2 + (1 - \lambda)r_{t-1}^2$ where $\lambda$ is usually chosen to be between 0.9 and 1.

# 5: Finding Trades with Positive Expected Value

A rule is a defined way to monetize an edge, whereas the edge is an observed phenomenon that could potentially be profited from in many ways. Edges can persist.

## Crowding

Generally, the "Crowding" idea encapsulates a number of situations in which the unintentional coordinated actions of traders create feedback effects.

*Baltas, N. 2019. “The Impact of Crowding in Alternative Risk Premia Investing.” Financial Analysts Journal. SSRN: 33603350* classifies strategies into either convergennce or divergence strategies. Convergence strategies, such as value investing, have a natural target price. This creates a stabilizing effect. Divergence strategies, such as trend-following, have no anchoring price. The more these assets go up, the more are bought. This creates a destabilizing feedback effect. In short term this will help profits, but it will eventually lead to a bubble and subsequent crash.

Baltas postulated that crowdedness could be quantified by looking for increases in co-movements of assets beyond their beta. In the presence of a natural target, crowding stabilized markets. If there was no natural target value, crowding was a destabilizing factor.

* It is never good to be so leveraged that you are forced to exit a trade.
* ETNs are more dangerous than ETFs.
* It is dangerous to be in a strategy or product that has to make trades and has no discretion in doing so.
* It is even more dangerous when other people know what you need to do and can push the market against you.
* The day you say you have to do something, you're screwed. Because you are going to make a bad deal.

## Trading Strategies

I assign each edge a rating on a scale of 1 to 3 with 3 being the best. This is a subjective score based on the amount of empirical and theoretical support, the longevity of the effect, and the volatility of the associated trade results. I also suggest ways to implement each effect. These are also subjective.

### Level 3 Strategies

These strategies are based on effects that have been well documented over either many markets or time periods. They also have convincing theoretical bases.

***
#### Implied Volatility Term Structure as a Predictor

It is well known that the term structure of commodity futures is a predictor of future returns (*Gorton, G., F. Hayashi, and K. Rouwenhorst. 2013. “The Fundamentals of Commodity Futures Returns.” Review of Finance 17: 35–105*).

When a commodity has a term structure in <u>contango (long-dated futures more expensive than short-dated futures)</u>, it is profitable to short the futures. And when futures are in <u>backwardation (long dated futures cheaper than short-dated futures)</u>, it is profitable to buy futures. Essentially, the cash price is a better predictor of futures prices than futures are.

The effect also exists for VIX futures. Simon and Campasano (2014) present evidence that the futures can be predicted by looking at the basis, i.e. if the futures are trading over the index, VIX, the futures will tend to fall, and if the futures are trading below the index, they will tend to rise (*Simon, D., and J. Campasano. 2014. “The VIX Futures Basis: Evidence and Trading Strategies.” The Journal of Derivatives 21: 54–69*).

The VIX term structure is usually in <u>contango at low VIX levels</u> and in <u>backwardation when the VIX is high</u>. This means the term-structure anomaly is often in conflict with the mean reversion of the VIX. Mean reversion would enourage you to buy futures at low VIX levels even in the presence of contango. However, it seems that the term structure is usually a stronger predictor of futures returns. The exception would be when the VIX is at very high levels.

This is probably why the term-structure effects fails to apply to the cross-section of equity option returns. Vasquez (2017) found that long straddle portfolios with high contango in the volatility term structure outperfortmed straddle portfolios with low contango or backwardation (*Vasquez, A. 2017. “Equity Volatility Term Structures and the Cross Section of Option Returns.” Journal of Financial and Quantitative Analysis 52: 1–28*). **[TODO:] Review the study, code the strategy, and try. Have noticed contango/backwardation in IV vs DTE charts in Augen notebook.**

The effect has existed in commodity futures for at least as long as we have had data to look at. This points to it being a mispriced risk premium.

**<u>Trading Strategy</u>**

<u>Sell VIX futures or index options when the term structure is in contango. Buy VIX futures or index options when the term structure is in backwardation.</u> **[TODO:] Identify appropriate VIX Index Options, code the startegy, and try.**

***
#### Options and Fundamental Factors

It is now well accepted that certain factors predict future stock returns (smart beta). Value stocks outperform growth stocks. Small cap stocks outperform large cap stocks. Low beta stocks outperform high beta stocks. High momentum stocks outperform low momentum stocks. High quality stocks outperform low-quality stocks What is less well known is that similar fundamental factors also predict volatility returns.

Cao et al (2015) found that long volatility positions profits increase with size, momentum, profitability and decrease with cash holdings, analysis forecast dispersion, and new issuance.

**<u>Trading Strategy</u>**

* On Friday of each week, rank all stocks in S&P100 or other Index, excl. financial companies, according to the valuation metrics (P/E, P/B, MarketCap, P/CF, D/E, RoE, RoA)
* Sell second monthly ATM starddles on top quartile and buy straddles for bottom quartile.
* This portfolio is held for a week, then the process is repeated.
* Try to be long volatility in stocks with these characteristics: low P/E, low P/CF, High market cap, High RoE, High RoA, and High D/E. Create a combined ranking consisting of 1/6th the P/E ranking plus 1/6th the P/CF ranking and so on.

*Reference*

* Cao, J., B. Han, Q. Tong, and X. Zhan. 2015. “Option Return Predictability,” 27th Annual Conference on Financial Economics and Accounting Paper, Rotman School of Management Working Paper No. 2698267.

**Implementation**

Further research and test Options and Fundamental factors before implementing the strategy.

***
#### Post-Earnings Announcement Drift (PEAD)

Post-earnings announcement drift (PEAD) is the tendency for a stock to continue to move in the direction caused by an earnings surprise. Ball and Brown (1968) found evidence that stock returns continue to drift in the same direction as unexpected earnings, i.e. stock prices tend to increase after earnings announcements with positive earnings surprises and decrease with negative earnings surprises. What was unexpected was that the outperformance didn't happen abruptly but accumulated over three months. The stock prices did incorporate the news but did so slowly. In fact, the prices reacted so slowly that the effect was tradable.

In the 1980s the consensus of analyst forecasts became available. This enabled the magnitude of the earnings surprise to be defined as the percentage difference between the reported EPS and the consensus forecast. Sorting stocks on the basis of this surprise gave the same result. Further, it seems that the effect of the earnings report isn't limited to the headline EPS number. The momentum created by the earnings report is a follow through of the initial price move based on the entire report, rather than being due to any single accounting number.

It is most pronounced in firms that report after their direct competitors and have earnings that outperform those firms by a surprising amount (Lee, Y. 2017). Long-short stock portfolios based on the size of surprises have been shown to return 9% to 27% annually.

Almost all of the drift occurs within 9 months for small firms and within 6 month for large firms. And a disproportionate amount of the return occurs within 5 days of the earnings announcement.

DellaVigna and Pollet (2009) postulate that this effect is even more pronounced on Fridays when investor attention is at its lowest.

**<u>Trading Strategy</u>**

Because IV tends to be comparatively cheap after earnings releases, although this isn't enough to profit from delta-neutral volatility strategies (Chung, S., and H. Lewis. 2017), this is an effect that can be traded from the long side.

For stocks with positive earnings surprises, being long a short-dated 40 delta/ 10 delta call spread is usually a cheap way to leverage PEAD. For bearish positions a 50 delta/20 delta put spread achieves the same effect while also selling one of the most expensive parts of the IV curve.

An alternative approach is to sell a covered call or put. Although the variance premium being collected won't be significant, this method is guaranteed to profit if any drift occurs at all.

*References*

* Ball, R. J., and P. Brown. 1968. “An Empirical Evaluation of Accounting Income Numbers.” Journal of Accounting Research 6: 159–178.
* Lee, Y. 2017. “Market Reactions to Unexpected Relative Earnings Performance.” Asia-Pacific Journal of Accounting and Economics 24: 339–357.
* DellaVigna, S., and J. Pollet. 2009. “Investor Inattention and Friday Earnings Announcements.” Journal of Finance 64: 709– 749.
* Chung, S., and H. Lewis. 2017. “Earnings Announcements and Option Returns.” Journal of Empirical Finance 40: 220–235.

**Implementation**

During the earning season, follow the following strategies in Steps:

1. Trading Equity Options over Earnings Announcement
    * Long Straddle, 5-10 days before earnings and close the day before earnings.
    * Short Straddle, the day of or day before earnings
2. PEAD
    * Long Call/Put Spread or Covered Call or Naked Put.

*[TODO:] Try on stocks that reported earnings recently.*

### Level 2 Strategies

These strategies may be lacking in empirical evidence or theoretical justification or they could provide more limited opportunities in some way.

***
#### Trading Equity Options over Earnings Announcement

The front-expiration IV will almost always rise significantly in the weeks before a company's earnings release. After the release the IV will collapse (Patell and Wolfson, 1979). Buying options before the release is profitable (Chung and Lewis, 2017). And even in cases when the options do not increase in price, the rise in IV gives a very cheap long position where the vega profits negate most of the time decay.

Gao et. al (2017) bought delta-neutral straddles 3 days before the earning announcement. Selling a day before the release gave a return of 1.9%, selling on the release day returned 2.60% and holding for a day after earnings had a return of 1.98%. Notably, holding the straddle for a full day after earnings had a lower return than selling on the release date. This indicates that the return on the straddle is negative after earnings are announced. As close as possible before earnings, sell the front straddle and hope that the collapse in IV compensates for the movement of the underlying.

However, it seems that there is a small tendency for this strategy to fare poorly with high PE stocks (He et al., 2010). However, the difference is so weak it can probably be ignored.

However, there is a link between the dispersion of analyst estimates, defined as (high estimate - low estimate) / average estimate, and profitability. Trading the effect on stocks with high dispersion is more profitable. Other signifiers of uncertianty are with higher historical volatilities, larger historical earning surprises, and more volatile earnings surprises. The strategies also perform better in these cases. The strategy was more profitable in stocks that were covered by fewer analysts.

The fact that pre-earnings long straddles are profitable makes it unlikely that this effect is due to risk premium. Separating the effect into the pre-earnings IV increase and the post-earnings IV collapse and stock price adjustment makes finding possible explanations easier. The profitability of the post-earnings straddle sales could be due to ambiguity aversion.

The short side of this strategy hasn't done well recently (2016-2018). Usually, selling options before situations of uncertainty is a good trade.

**<u>Trading Strategy</u>**

On the long side, buy the shortest-dated straddles that span the earning date. Or if the smile is relatively flat, strangles are a higher risk/reward play.

On the short side, sell the shortest-dated straddles that span the earnings date. After the earnings release, either buy the straddle back (generally preferable if trade is winner), or if stock has a big move, hedge the losing leg as 100 delta. Stocks in which the earnings number is outside of the most extreme estimates are most likely to not reverse.

*Reference*

* Patell, J., and M. Wolfson. 1979. “Anticipated Information Releases Reflected in Call Option Prices.” Journal of Accounting and Economics 1: 117–140
* Chung, S., and H. Lewis. 2017. “Earnings Announcements and Option Returns.” Journal of Empirical Finance 40: 220–235.
* Gao, C., Y. Xing, and X. Zhang. 2017. “Anticipating Uncertainty: Straddles around Earnings Announcements.” SSRN: 2204549.
* He, W., Y. Lee, and P. Wei. 2010. “Do Option Traders on Value and Growth Stocks React Differently to New Information?” Review of Quantitative Finance and Accounting 34: 371–381.


**Implementation**

1. Buy front expiration ATM straddle 10 days before the earning announcement.
2. Close the long ATM straddle position on the day of earnings or the day before earnings. Leave the losing leg open to act as Long side of the spread for one side of the Short Straddle.
3. Sell front expiration ATM straddle the day before or on the day of earnings.
4. After earnings, buy back the straddle or hedge the losing leg as 100 delta.


***

#### The Overnight Effect

The index variance premium is realized overnight (Muravyev and Ni, 2018). Returns to index options are negative, about -0.7% a day in terms of actual premium decay for the S&P, all of this comes from overnight decay. Delta-hedged option returns are -1% overnight, and intraday returns are positive at +0.3% per day. The anomaly holds for options of all maturities and moneyness and also equity options.

Overnight returns are less volatile than intraday returns. The risk is much higher during the untraded or illiquid hours.

**<u>Trading Strategy</u>**

Being short options overnight and flattening during the day is impractical due to transaction costs. Instead, sell very short term options that include as many overnight periods as possible. For example, instead of selling daily options on the morning of expiration, sell them the night before.

*Reference*

* Muravyev, D., and X. Ni. 2018. “Why Do Option Returns Change Sign from Day to Night?” SSRN: 2820264.

**Implementation**

[TODO:] Evaluate Selling options expiring the next day (overnight effect), on Friday expiring Monday (weekend effect), or before holidays expiring the day after holidays.


***
#### FOMC and Volatility

Fernandez-Perez et al. (2017) show that the VIX and VIX Futures drop significantly after the announcements of the meeting. This decline persists for about 45 minutes after the announcement. The VIX declines by about 3% on announcement days, whereas nearest term VIX futures contract declines about 1.4% around the announcement. Nikkinen and Sahlstrom (2004) showed that the VIX index declined after FOMC announcements, CPI and PPI releases, and the Non-farm Payroll report. Gospodinov and Jamali (2012) found both implied and realized volatility dropped after the release and any initial market reaction.

Trading the FOMC related IV collapse with VIX futures avoids the problem of IV collapse partially offset by increase in realized volatility encountered by stocks after earnings announcement. We can trade an implied effect with an implied contract and have no realized volatility exposure.

Equity market returns have been about 30 times larger on announcement days than on normal days. Surprisingly most of these excess returns occur before the announcement (Lucca and Moench, 2011). Stocks tend to rally strongly in the day and a half before the announcement.

It seems that this is a market-wide version of the pre-earnings drift seen on inidividual equities. As in that case, investors buy stocks because of an attention-grabbing event (Lamont and Frazzini, 2007).

**<u>Trading Strategy</u>**

Buy S&P500 futures two days before the FOMC release. Sell these before the announcement and at the same time sell VIX futures. Cover the short VIX futures 45 minutes after announcement.

*Reference*

* Fernandez-Perez, A., B. Frijns, and A. Tourani-Rad. 2017. “When No News Is Good News–The Decrease in Investor Fear after the FOMC Announcement.” Journal of Empirical Finance 41: 187– 199.
* Nikkinen, J., and P. Sahlström. 2004. “Impact of the Federal Open Market Committee's Meetings and Scheduled Macroeconomic News on Stock Market Uncertainty.” International Review of Financial Analysis 13: 1–12.
* Gospodinov, N., and I. Jamali. 2012. “The Effects of Federal Funds Rate Surprises on S&P500 Volatility and Volatility Risk Premium.” Journal of Empirical Finance 19: 497–510.
* Lucca, D., and E. Moench. 2011. “The pre-FOMC Announcement Drift.” Staff Report, No. 512, Federal Reserve Bank of New York.
* Lamont, O., and A. Frazzini. 2007. “The Earnings Announcement Premium and Trading Volume.” NBER Working Papers 13090, National Bureau of Economic Research.

**Implementation**

[TODO:] Evaluate the strategy of buying SPY/QQQ/UPRO/TQQQ ETF options two days before FOMC release and selling before the announcement. Same time sell appropriate VIX ETF and cover 45 minutes after announcement.


***
#### The Weekend Effect

Equity options decay more than expected over weekends. Jones and Shemesh (2017) found the average US equities option return over the weekend was -0.62% while the returns for all other days were +0.18% a day. This all seems to be true over the long weekends and holidays. Returns on equity options are negative whenever the market is closed. This effcet is significant.

**<u>Trading Strategy</u>**

On a Friday, sell the options that expire next Monday. In general, time any short-volatility strategies to include as many non-trading periods as possible.

*Reference*

* Jones, C., and J. Shemesh. 2017. “Option Mispricing around Nontrading Periods.” The Journal of Finance 73: 861–900.

**Implementation**

[TODO:] Evaluate the strategy of selling options on Friday with nearest expiration following week.

***
#### Volatility of Volatility Risk Premia

Options on products with high volatility of volatility tend to be overpriced.

* Options on stocks with high volatility of volatility are overpriced relative to options on stocks with low volatility of volatility.

* The VIX tend to decline (rise) with very high (low) values of the VVIX index.

The relationship between high VVIX (the model-free IV derived from VIX options) and subsequent lower VIX levels is very strong.

**<u>Trading Strategy</u>**

When VVIX reaches extremely high, 1-year 90th percentile, (low, 1-year 10th percentile) levels either sell (buy) VIX futures or sell (buy), and dynamically hedge, S&P 500 straddles.

If the VVIX crossed above the rolling 1-year 90th percentile of VVIX, SELL the VIX (through VIX futures) and hold until VVIX reaches its rolling 1-year median. BUY the VIX (through VIX futures) when VVIX crossed below the rolling 1-year 10th percentile of VVIX and hold until VVIX reaches its rolling 1-year median.

Different look back periods or moving averages instead of medians can also be used.

*References:*

* Park, Y. 2015. “Volatility-of-Volatility and Tail Risk Hedging Returns.” Journal of Financial Markets 26: 38–63.
* Huang, D., C. Schlag, I. Shaliastovich, and J. Thime. 2018. “Volatility of Volatility Risk.” Journal of Financial and Quantitative Analysis 53: 1–63.
* Cao, J., A. Vasquez, X. Xiao, and X. Zhan. 2018. “Volatility Uncertainty and the Cross Section of Option Returns.” SSRN: 3178263. [TODO:] Read.
* Ruan, X. 2017. “Cross Section of Option Returns and Volatility-of- Volatility.” SSRN: 3055982.

**Implementation**

[TODO:] Evaluate the strategy with different VIX based ETFs and SPY Straddles.

### Level 1 Strategies

Trades based on market inefficiencies will be most profitable when the evidence for them is still underwhelming. Many inefficiencies will not survive long enough to reach higher confidence level.

***
#### Earnings-Induced Reversals

Earnings-induced reversals are the tendency of stocks that have drifted a lot before their earnings announcement to reverse the pre-announcement drift when the news breaks. So and Wang (2014) created a trading strategy that shorted stocks with high market-adjusted returns ib the period from 4 days before earnings through 2 days before earnings and went long those stocks with the worst pre-earnings market adjusted returns. Liquidating the portfolio on the close after earnings, they found this portfolio made 145 bps compared to 22 bps earned by a similarly constructed non-earnings periods.

A similar study by Jansen and Nikiforov (2016) shows Simply fading stocks with large percentage moves in the week before earnings would have averaged 1.3% over 2-day periods, potentially due to investor overreaction in the pre-earnings period.

**<u>Trading Strategy</u>**

I'm more confident in the collapse of IV when earnings are released than I am of this reversal effect. So, when I trade both of these effects together I sell a straddle but shade the delta if I want to also bet on the reversal.

*References:*

* So, E., and S. Wang. 2014. “News-Driven Return Reversals: Liquidity Provision ahead of Earnings Announcements.” Journal of Financial Economics 114: 20–35.
* Jansen, I., and A. Nikiforov. 2016. “Fear and Greed: A Returns- Based Trading Strategy around Earnings Announcements.” Journal of Portfolio Management 42: 88–95.

**Implementation**

[TODO:] Evaluate following strategy:

1. Open a long straddle 1 week to 1 month before earnings.
2. Close the profitable leg a day before earnings. Keep the unprofitable leg open.
3. Open a short straddle a day before earnings. The unprofitable leg from long straddle act as spread with the short straddle leg on the same side.
4. Close short straddle positions the day after earnings.
5. If the stock moves in the direction of unprofitable leg from long straddle, leave it open else close it too after earnings.

***
#### Pre-Earnings Announcement Drift

Pre-earnings announcement drift is the tenedency of stocks to move in the direction of any earning-related abnormal returns experienced by stocks in the same industry that reported earlier. The earnings information for the earliest announcing firm within as industry predicts both the earnings surprise and the returns of other firms within the industry.

The drift begins as the results from the early announcers are reported and continues up until the later announcing stock releases its earnings. If earlier reporting stocks all rally, we would expect later reporting stocks to also rally just due to industry exposure.

Pre-announcement drift is a separate effect.

**<u>Trading Strategy</u>**

As we also expect IV to increase in the time leading to the earnings release, any long volatility directional strategy would be appropriate. For example, if we expect a rally, we could buy a call or call spread. My preference is for a 50 delta/20 delta, 1-month call spread.

*References:*

* Ramnath, S. 2002. “Investor and Analyst Reactions to Earnings Announcements of Related Firms: An Empirical Analysis.” Journal of Accounting Research 40: 1351–1376.
* Easton, P., G. Gao, and P. Gao. 2010. “Pre-earnings Announcement Drift.” SSRN: 1786697.

**Implementation**

[TODO:] Evaluate following strategy on stocks already owned:

1. Find competitor stocks in same industry that report before the owned stock.
2. If the competitor reports negative earning, open a 50 delta/20 delta 1-month Put debit spread, a day after competitor's earning report. Not sure about DTE of the position, may also consider the expiry just after earnings.
3. Close the position a day before owned stock's earnings.

## Summary

* It is worthwhile to search SSRN periodically to find new trading ideas.

* Many volatility trading edges involve selling options in situation of uncertainty, an extra situational variance premiun.

* Because of the variance premium, long volatility strategies are unlikely to have as much edge as that involve selling options.